# Design Web Crawler

## Functional
- Crawl all the web. (HTML only)
- Consider HTTP protocol only.

## Capacity

### Assume
- 1 billion web sites, which is in turn 15 billion HTML pages.
- Then, 15B / (4 weeks * 7 days * 86400 sec) = 6200 pages/sec.
- Page size is 100KB on average with 500 bytes metadata.

### Storage
- 15B * (100KB + 500) = 1.5 petabytes.

## Design

### In each iteration
- Pick a URL from unvisited URL list.
- Determine the ID address of the hostname.
- Download the document.
- Parse document contents to look for new URLs.
- Add new URLs to unvisited URL list.
- Process downloaded document.

<img src="img/web-crawler1.png" style="width:800px;height:600px;">

### URL frontier
- Contains all the remaining URLs to download.
- Prioritize which URLs should be downloaded first.
- Use BFS implemented by a queue.
- Distributed into multiple servers.
    - Each server maintains many queues such that each thread gets a queue.
    - Hash table maps carnonical hostname to thread number that run the queue.
- Due to huge size of URLs, need to store URLs into a disk.
    - Enqueue buffer, once filled, is dumped into disk.
    - Dequeue buffer keeps cache of URLs to be visited. It periodically reads from disk to fill the buffer.
    
### HTML fetcher
- Downloads documents corresponding to givne URL using protocol like HTTP.

### Document input stream
- Caches the entire document contents.
- Prevents downloading the same document again.

### Document de-dup
- Checks if document is duplicate.
- Calculates 64-bit checksum of every processed document using MD5 or SHA and store it in DB.

### URL filter
- Blocks some websites so that crawler can ignore them.

### DNS resolver
- Caches DNS lookup results.

### URL de-dup
- Check if URL is duplicates. (Multiple URL linking to the same document)
- Store checksum of URLs in carnonical forms into DB.

## Fault tolerance
- Consistent hashing

## Data partitioning
- Distribute based on hostname which contains
    - URLs to visit.
    - URL checksum.
    - Document checksum.